In [1]:
# import requirements 
import requests
import re
import pandas as pd
import geopandas as gpd
from arcgis.gis import GIS
import os
import numpy as np
from arcgis.features import FeatureLayerCollection
from arcgis.features import FeatureSet, GeoAccessor
from arcgis.geometry import Geometry

In [2]:
#Import external files
governorate_municipality = gpd.read_file("UNEP_Gaza_Governorate_Municipality_Singleparts_POINTS.gpkg")
governorate_municipality = governorate_municipality.to_crs(epsg=4326)
governorate_municipality.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [5]:
governorate_municipality

,OBJECTID,Neighborhood,Municipality,Governorate,Shape__Area,Shape__Length,ArabicName,governate_name,municipality_name,neighborhood_name,Location_gdf,Gov_Mun,geometry
0,1,Al Fukhkhari,Al Fukhkhari,Khan Younis,2.303302e+05,2161.057834,None,khan_younis,al_fukhkhari,al_fukhkhari,(Khan Younis / Al Fukhkhari)/Al Fukhkhari,Khan Younis_Al Fukhkhari,POINT (34.33681 31.28982)
1,2,Abu Mgheiseb,Wadi as Salqa,Deir Al-Balah,1.875443e+06,6378.072751,None,deir_al_balah,wadi_as_salqa,abu_mgheiseb,(Deir Al-Balah / Wadi as Salqa)/Abu Mgheiseb,Deir Al-Balah_Wadi as Salqa,POINT (34.3612 31.39191)
2,3,Al Qaraya al Badawiya al Maslakh,Um Al-NASER,North Gaza,2.287709e+06,6929.079613,None,north_gaza,um_al_naser_,al_qaraya_al_badawiya_al_maslakh,(North Gaza / Um Al-NASER)/Al Qaraya al Badawiya al Maslakh,North Gaza_Um Al-NASER,POINT (34.52588 31.56258)
3,4,Al Basateen,Al Musaddar,Deir Al-Balah,1.565272e+06,7815.068901,None,deir_al_balah,al_musaddar,al_basateen,(Deir Al-Balah / Al Musaddar)/Al Basateen,Deir Al-Balah_Al Musaddar,POINT (34.38568 31.41439)
4,5,Al Attarta,Beit Lahiya,North Gaza,4.938257e+06,12707.141053,None,north_gaza,beit_lahiya,al_attarta,(North Gaza / Beit Lahiya)/Al Attarta,North Gaza_Beit Lahiya,POINT (34.50268 31.56083)
5,6,Hai Al Amal,Beit Hanun,North Gaza,8.140580e+06,12980.393705,None,north_gaza,beit_hanun,hai_al_amal,(North Gaza / Beit Hanun)/Hai Al Amal,North Gaza_Beit Hanun,POINT (34.53408 31.53146)
6,7,Other,Al Bureij,Gaza,4.641780e+06,14448.289435,None,gaza,al_bureij,other,(Gaza / Al Bureij)/Other,Gaza_Al Bureij,POINT (34.41727 31.44893)
7,8,Other,Madinat Ezahra,Gaza,8.744942e+06,17099.858596,None,gaza,madinat_ezahra,other,(Gaza / Madinat Ezahra)/Other,Gaza_Madinat Ezahra,POINT (34.40827 31.47446)
8,9,Ansar,Az Zawayda,Deir Al-Balah,7.660522e+06,18064.708557,None,deir_al_balah,az_zawayda,ansar,(Deir Al-Balah / Az Zawayda)/Ansar,Deir Al-Balah_Az Zawayda,POINT (34.36975 31.43705)
9,10,Other,Khuza'a,Khan Younis,8.009019e+06,20369.119178,None,khan_younis,khuza_a,other,(Khan Younis / Khuza'a)/Other,Khan Younis_Khuza'a,POINT (34.3581 31.31072)


In [ ]:
#Read this to know more about how Kobo handles synchronous exports and where to find the right url
#https://support.kobotoolbox.org/synchronous_exports.html

#### FETCH KOBO DATA

# Authentication credentials
KOBO_USERNAME = "guilherme_iablonovski"
KOBO_PASSWORD = "k0b0Senha"
export_token = "f72722c6ffe97db14144325853528a4b7a1c059b"

#Fuel Request API endpoint
kobo_api_url = "https://kf.kobotoolbox.org/api/v2/assets/aYyrjnacEdnBdZnXaxMcG5/export-settings/eskV2HM9SKSUw6pB93aZqWK"

In [ ]:
def getKoboForm(url):
    # Create a session with authentication
    session = requests.Session()
    session.auth = (KOBO_USERNAME, KOBO_PASSWORD)

    # Construct the export URL
    kobo_export_url = f"{url}/data.xlsx"#?format=csv&token={export_token}"

    # Fetch data using the authenticated session
    response = session.get(kobo_export_url)

    # Check if the request was successful
    if response.status_code == 200:
        # Load data into a pandas DataFrame
        kobo_data = pd.read_excel(response.content)
        kobo_data.head()
        return kobo_data
    else:
        print("Error:", response.status_code)
        
kobo_data = getKoboForm(kobo_api_url)

In [ ]:
kobo_data

In [ ]:
# Identifica as colunas que começam com "Please specify the municipality"
cols = [c for c in kobo_data.columns if c.startswith("Please specify the municipality")]

# Cria uma lista para armazenar os registros explodidos
records = []

# Itera sobre cada linha e coluna dessas
for _, row in kobo_data.iterrows():
    base_data = {col: row[col] for col in kobo_data.columns if col not in cols}  # mantém outras colunas

    for col in cols:
        val = row[col]
        if pd.isna(val):
            continue

        # Extrai o nome entre parênteses
        match = re.search(r'\((.*?)\)', col)
        governo = match.group(1).strip() if match else ''

        # Divide as respostas usando ". " como separador
        municipios = re.split(r'(?<=\.)\s+', str(val).strip())

        # Limpa e forma a nova lista com "Governo_Município"
        for m in municipios:
            m = m.strip().rstrip('.')
            if m:
                new_row = base_data.copy()
                new_row['Gov_Mun'] = f"{governo}_{m}"
                records.append(new_row)

# Cria o novo DataFrame com as linhas expandidas
kobo_data_exploded = pd.DataFrame(records).reset_index(drop=True)

In [ ]:
kobo_data_exploded

In [ ]:
#### INITIAL CLEANING

#Standardize data for municipalities without neighborhoods

def standardize(df):

    kobo_data_0 = df.replace(0, None)  
    return kobo_data_0

kobo_data_exploded = standardize(kobo_data_exploded)

# Faz o join com o gdf, mantendo todos os municípios originais

gdf_filtrado = governorate_municipality.merge(kobo_data_exploded, on='Gov_Mun', how='right')


In [ ]:
gdf_filtrado.plot()

In [4]:
pd.set_option("display.max_columns", None)  # mostra todas as colunas
pd.set_option("display.max_rows", None)     # mostra todas as linhas
pd.set_option("display.width", None)        # evita quebra de linha automática
pd.set_option("display.max_colwidth", None) # evita truncar valores longos

In [ ]:
gdf_filtrado

In [ ]:
#Handle columns so they are not modified by arcgis when uploaded
def makeArcGISfriendly(df):
    df.columns = df.columns.str.replace(r"[ ]", "_", regex=True)
    df.columns = df.columns.str.replace(r"[.]", "_", regex=True)
    df.columns = df.columns.str.replace(r"[?]", "_", regex=True)
    df.columns = df.columns.str.replace(r"[']", "", regex=True)
    df.columns = df.columns.str.replace(r"[(]", "", regex=True)
    df.columns = df.columns.str.replace(r"[)]", "", regex=True)
    df.columns = df.columns.str.replace(r"[[]", "", regex=True)
    df.columns = df.columns.str.replace(r"[]]", "", regex=True)
    df.columns = df.columns.str.replace(r"[%]", "_", regex=True)
    df.columns = map(str.lower, df.columns)
    
    # Remove leading underscores
    df = df.rename(columns={
        '_id':'f_id',
        '_uuid':'f_uuid',
    })
    df.columns = df.columns.str.lstrip('_')
    
    #Truncate and ensure uniqueness
    seen = {}
    final_cols = []
    max_length = 31
    for col in df.columns:
        base = col[:max_length]
        new_col = base
        i = 1
        while new_col in seen:
            suffix = f"_{i}"
            trim_len = max_length - len(suffix)
            new_col = base[:trim_len] + suffix
            i += 1
        seen[new_col] = True
        final_cols.append(new_col)
    
    # Step 4: Apply to DataFrame
    df.columns = final_cols
    return df

gdf_exploded = makeArcGISfriendly(gdf_filtrado)

In [ ]:
def simplifyGeometries(df):
    df['geometry'] = df['geometry'].centroid
    return df
    
gdf_exploded = simplifyGeometries(gdf_exploded)

seen = set()
unique_flags = []

for _, row in gdf_exploded.iterrows():
    if row['f_id'] in seen:
        unique_flags.append('')
    else:
        unique_flags.append('yes')
        seen.add(row['f_id'])

gdf_exploded['unique_'] = unique_flags

In [ ]:
#### UPDATE DATA IN ARCGIS

#Connect to the ArcGIS Enterprise portal

AGOL_USERNAME = 'h.partow'
AGOL_PASSWORD = 'R&Runit2024'
gis = GIS('https://wesrmapportal.unep.org/portal/', AGOL_USERNAME, AGOL_PASSWORD)

# Access the feature-layer through its URL
file = "https://wesrmapportal.unep.org/arcgis/rest/services/Hosted/DWG_Gaza_Fuel_Requests_Responses/FeatureServer"

# Access the feature-layer through its URL
fuel_file = 'DWG_Gaza_Fuel_RequestsResponses'


In [ ]:
def searchArcgis(keyword):
    if not isinstance(keyword, str):
        raise TypeError(f"Esperado 'str' em 'keyword', recebido {type(keyword)} → {keyword}")
    results = gis.content.search(f'title:\"{keyword}\"', item_type="Feature Layer")
    if not results:
        print("Nenhum resultado encontrado para", keyword)
        return None
    results_sorted = sorted(results, key=lambda x: x.modified, reverse=True)
    return results_sorted[0]

search_fuel = searchArcgis(fuel_file)

In [ ]:
def updateFeature(search_results, gdf):

    item = gis.content.get(search_results.id)

    flc = FeatureLayerCollection.fromitem(item)
    layer = flc.layers[0]  # Assuming you're working with the first layer


    # Step 1: Truncate features
    truncate_result = layer.manager.truncate()
    print("Truncate result:", truncate_result)

    # Keep only matching columns
    expected_fields = [f["name"] for f in layer.properties.fields if f["name"] != "fid"]
    gdf = gdf[[col for col in gdf.columns if col in expected_fields or col == "geometry"]]

    gdf = gdf.to_crs(epsg=4326)

    sedf = GeoAccessor.from_geodataframe(gdf)

    fs = FeatureSet.from_dataframe(sedf)

    result = layer.edit_features(adds=fs.features)
    print("Result:", result)
    return(result)

updateFeature(search_fuel, gdf_exploded)